# MEBDI Competition Submission: Lejvi Dalani & Hasan Tosun
## Estimating Income From Worker Characteristics

This is the code for the second part of the MEBDI competition. The goal of the competition was to estimate wage income using variables from the Current Population Survey (CPS), and improve Root-Mean-Square Error relative to the Mincer Regression model of income. In the second part of the competition, all variables in CPS  were allowed, as long as they were not directly related to realized income. 

# 1. Prerequisites

In [17]:
import pandas as pd

from sklearn.model_selection import train_test_split

import sklearn as sk

import xgboost as xgb #requires pip install xgboost

import lightgbm as lgb

from sklearn.model_selection import cross_val_score

from sklearn import metrics

import numpy as np

from sklearn.model_selection import RandomizedSearchCV, KFold

from vecstack import stacking #pip install vecstack

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import GradientBoostingRegressor

from sklearn import preprocessing

import tensorflow as tf

from tensorflow.keras import layers

from tensorflow.keras import optimizers

from tensorflow.keras.models import Model

from sklearn.linear_model import Lasso

import catboost

from catboost import CatBoostRegressor 

import os

import random



In [18]:
# Seed value
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_random_seed(seed_value)


# 2. The data

The data came from IPUMS CPS 2012. We provide a list variables in `var_list.csv` with the variable names and whether they are used in any part of the project, and a description of the 125 variables used in `Variables_Dalani_Tosun.xslx`. Please refer to the submission report `Mebdi_Dalani_Tosun.pdf` for a detailed description of the data selection process. All files mentioned on this script can be found in the repository.

In [19]:
df = pd.read_feather("data_part2.feather")
# df = pd.read_feather("data_part2.feather")
var_list = pd.read_csv("var_list.csv")

vars_dummy = var_list[var_list.decision == 'dummy']['var'].tolist()
vars_numeric = var_list[var_list.decision == 'numeric']['var'].tolist()
vars_entityembed = var_list[var_list.decision == 'entityembed']['var'].tolist()
vars_necessary = var_list[var_list.decision == 'necessary']['var'].tolist()
vars_drop = var_list[var_list.decision == 'drop']['var'].tolist()

vars_to_use = vars_dummy + vars_numeric + vars_entityembed + vars_necessary 

df = df[vars_to_use]

df.loc[:,'y'] = np.log(df.incwage)

Train = df[df.type == "train"].drop(vars_necessary, axis = 1).reset_index(drop=True)
Test = df[df.type == "test"].drop(vars_necessary, axis = 1).reset_index(drop=True)


Index_train = Train.index.to_numpy()
Index_test = Test.index.to_numpy()

Records_train = Train.to_records()
Records_test = Test.to_records()

# 3. Entity Embedding 

Variables occ (occupation) and occly (occupation last year) showed up as important features across
multiple feature selection algorithms we tried. However, both features had high dimensionality (more
than 500 categories in each), with very few observations for some categories in some of the samples.
Given how small our sample size was, we decided to reduce the dimensionality of the two variables using *entity embedding*, using a Neural Network. 

This allowed us to reduce the dimensionality of each of the occupation variables
to 23 each. The idea of entity embeddings is to create vectors in a smaller subspace (in this case in $R^{23}$),
instead of the sparse $R^{500+}$ space needed by dummy encoding. Each vector in $R^{23}$ corresponds to one
"unique" occupation. As a consequence, occ and occly were mapped into 46 columns. These 46 columns
were merged into our dataframe, and the raw occ and occly variables were dropped, in order to avoid
overfitting, since our train data had already seen the two variables.

In [20]:
print('Prepping Data for Entity Embedding')

features = vars_entityembed

#######################################################################
# There are categorical values that may show up only in test. So for now, need to join Train and Test Data,
# so as to create one hot encoding for all categories, and then right after separate.

# Create var to unmerge on later
Train.loc[:,'placeholder'] = 1
Test.loc[:,'placeholder'] = -1

# Merge
DF_embed = pd.concat([Train, Test])

DF_embed = DF_embed[features+['y']+['placeholder']]

# Impute values for occ and occly that show up only in train based on crosswalk
DF_embed['occ'].replace(1660, 1610, inplace = True)
DF_embed['occ'].replace(5200, 5165, inplace = True)
DF_embed['occ'].replace(7440, 7430, inplace = True)
DF_embed['occ'].replace(7550, 7560, inplace = True)
DF_embed['occ'].replace(8840, 8830, inplace = True)
DF_embed['occ'].replace(8850, 8830, inplace = True)
DF_embed['occ'].replace(8860, 8830, inplace = True)


DF_embed['occly'].replace(1660, 1610, inplace = True)
DF_embed['occly'].replace(5200, 5165, inplace = True)
DF_embed['occly'].replace(5210, 5165, inplace = True)
DF_embed['occly'].replace(7550, 7560, inplace = True)
DF_embed['occly'].replace(8840, 8830, inplace = True)

#Label Encode occ and occly: Turning everything into numbers basically    
for col in features:
    encoder = preprocessing.LabelEncoder()
    DF_embed.loc[:,col] = encoder.fit_transform(DF_embed[col].astype(str).values)

DF_embed = DF_embed[features +['placeholder','y']] 
    
Placeholder = DF_embed['placeholder']
Train_embed = DF_embed[DF_embed.placeholder == 1].drop("placeholder", axis = 1).reset_index()
Test_embed = DF_embed[DF_embed.placeholder == -1].drop("placeholder", axis = 1).reset_index()


## Create model

def create_model(df, categorical_columns, num_features_param):
    inputs = []
    outputs = []
    for c in categorical_columns:
        num_unique_vals = int(df[c].nunique())
        embed_dim = int(min(np.sqrt(num_unique_vals) + 8, num_features_param)) # embed dimension
        inpt = layers.Input(shape=(1, ))
        outt = layers.Embedding(num_unique_vals, embed_dim, name = c)(inpt) 
        # Can apply dropout here to introduce some regularization
        outt = layers.Reshape(target_shape = (embed_dim, ))(outt)
        inputs.append(inpt)
        outputs.append(outt)
    X = layers.Concatenate()(outputs)
    X = layers.Dense(2000, activation = 'relu')(X)
    X = layers.Dense(10, activation = 'relu')(X)
#     X = layers.Dropout(.2)(X)
    y = layers.Dense(1, activation ='linear')(X)
    model = Model(inputs = inputs, outputs = y)
    return model

print('Running Neural Network on entity embedding for occ and occly')
# Run and compile model
min_features = 23 
model = create_model(Train_embed, features, min_features)
adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.9999,                                     epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = adam, loss = 'mean_squared_error', metrics=['mse', 'mae'])
model.fit([Train_embed.loc[:, c].values for c in features], Train_embed.y.values,epochs=40, batch_size=4500, verbose = 0)


model.evaluate([Test_embed.loc[:, c].values for c in features], Test_embed.y.values, batch_size=300, verbose = 0)
result = model.predict([Test_embed.loc[:, c].values for c in features])
rmse = np.sqrt(np.mean((result-Test_embed.y.values.reshape(-1, 1))**2))


#### Get entity embeddings:
Occ_emb   = pd.get_dummies(DF_embed.occ) @ model.get_layer('occ').get_weights()[0]
Occly_emb = pd.get_dummies(DF_embed.occly) @ model.get_layer('occly').get_weights()[0]

Occ_col_list = ['occ' + str(x) for x in range(1,24)]
Occly_col_list = ['occly' + str(x) for x in range(1,24)]


Occ_emb.columns = Occ_col_list
Occly_emb.columns = Occly_col_list


X_embed   = pd.concat([Occ_emb, Occly_emb], axis = 1) 
X_embed   = pd.concat([X_embed, Placeholder], axis = 1)

# Store the embeddings
X_embed_train = X_embed[X_embed.placeholder == 1].drop("placeholder", axis = 1)
X_embed_test  = X_embed[X_embed.placeholder == -1].drop("placeholder", axis = 1)

print('Entity embedding for occ and occly done')

Prepping Data for Entity Embedding
Running Neural Network on entity embedding for occ and occly
Entity embedding for occ and occly done


# 4. Label Encoding and One-Hot Encoding

We decided to perform two types of encoding on the remaining categorical variables in our data:
LabelEncoding (randomly assigning numbers to each category within a variable) and OneHotEncoding
(creating dummies for each variable). These encodings were performed uniformly on all categorical
variables in order to create three dataframes: $DF_{enc}$, $DF_{onehot}^1$ and $DF_{onehot}^2$, where $DF_{onehot}^1$ and $DF_{onehot}^2$ are randomly column partitioned (of equal size) from $DF_{onehot}$, a dataframe resulting from
OneHot encoding all of the categorical variables. The partitions will become useful in the discussion
that follows regarding the algorithm.

In [21]:
#################### Label Encoded Section ########################
print('Prepping Data for Label Encoding')


vars_all_encoded = vars_dummy + vars_numeric 
vars_dummy_encoded = vars_dummy  
    

# Merge
DF = pd.concat([Train, Test])

DF = DF[vars_all_encoded +['placeholder','y']]

for col in vars_dummy_encoded:
    encoder = preprocessing.LabelEncoder()
    DF.loc[:,col] = encoder.fit_transform(DF[col].astype(str).values)
    
Train_encoded = DF[DF.placeholder == 1].drop("placeholder", axis = 1)
Test_encoded = DF[DF.placeholder == -1].drop("placeholder", axis = 1)

Train_encoded = pd.concat([Train_encoded, X_embed_train], axis = 1)
Test_encoded = pd.concat([Test_encoded, X_embed_test], axis = 1)

X_train_encoded = Train_encoded.drop("y", axis = 1).to_numpy()
y_train_encoded = Train_encoded["y"].to_numpy().reshape(-1, 1)

X_test_encoded = Test_encoded.drop("y", axis = 1).to_numpy()
y_test_encoded = Test_encoded["y"].to_numpy().reshape(-1, 1)




########################################################################
#################### One Hot Section ########################
print('Prepping Data for One Hot Encoding')

# Variable list for label encoded algorithms

vars_all_onehot = vars_dummy + vars_numeric 
vars_dummy_onehot = vars_dummy


# Merge
DF = pd.concat([Train, Test])

DF = DF[vars_all_onehot +['placeholder','y']]

#One hot encode all categorical variables    
DF_dummies = pd.get_dummies(data=DF, columns=vars_dummy_onehot)

    
Train_onehot = DF_dummies[DF_dummies.placeholder == 1].drop("placeholder", axis = 1)
Test_onehot = DF_dummies[DF_dummies.placeholder == -1].drop("placeholder", axis = 1)

# Merge in embedded occ and occly
Train_onehot = pd.concat([Train_onehot, X_embed_train], axis = 1)
Test_onehot = pd.concat([Test_onehot, X_embed_test], axis = 1)

X_train_onehot = Train_onehot.drop("y", axis = 1).to_numpy()
y_train_onehot = Train_onehot["y"].to_numpy().reshape(-1, 1)

X_test_onehot = Test_onehot.drop("y", axis = 1).to_numpy()
y_test_onehot = Test_onehot["y"].to_numpy().reshape(-1, 1)

y_train = y_train_onehot # also equal to y_train_encoded
y_test = y_test_onehot # also equal to y_test_encoded

############### Slice Columns
random_cols_one = list(range(X_train_onehot.shape[1]))
np.random.shuffle(random_cols_one)

# Do a 50/50 split
truncation_index_one = int(X_train_onehot.shape[1]*0.5)

# Get sub arrays
X_train_onehot_1 = X_train_onehot[:, random_cols_one[:truncation_index_one]]
X_train_onehot_2 = X_train_onehot[:, random_cols_one[truncation_index_one:]]

X_test_onehot_1  = X_test_onehot[:, random_cols_one[:truncation_index_one]]
X_test_onehot_2  = X_test_onehot[:, random_cols_one[truncation_index_one:]]

Prepping Data for Label Encoding
Prepping Data for One Hot Encoding


# 5. Models
We used **Stacked Generalization** as the algorithm. Our Stacked Generalization
algorithm consisted of two levels:
1. First Level
2. Meta Level (Second level)


## 5.1 First-Level layers

The First Level consisted of:

• Algorithms (tree-based) trained on the label encoded data, $DF_{enc}$:
 - A Random Forest algorithm, with off-the-shelf parameters
 - An XGB Regressor, which we tried to optimize using Random Search Cross Validation over the parameter space
 - A Gradient Boosting algorithm, with off-the-shelf parameterization, with a few tweaks to learning rate and depth of the trees.
 - An LGBM Regressor with off-the-shelf parameterization with smaller learning rate
• Algorithms (linear) trained on the OneHot encoded data, $DF_{onehot}^1$ and $DF_{onehot}^2$:
 - On each of the two column partitions of our data, the same Lasso regression was applied, for a total of two models. The regularization parameter was fine tuned based on cross validation using the training data.


The reason for the two types of algorithms mentioned above is the following: tree based algorithms
perform better on non-dummy variables, while linear models perform better with dummy-encoded
variables. Since there are disadvantages to both types of encoding with respect to training models, we
decided to control for both, and thus increase the diversity of our model. The predictions from all the
models in the 1st level were then stacked together.

In [22]:
############ Declare Models 1st level
print('Training 1st level models')

### XGB hyperparams
# Hyperparameters:
learner = 'gbtree' #could be set to gblinear, gbtree, dart
eval_metric_chosen = 'rmse'
depth = 8
estimators = 37 # number of trees
subsamples = 0.9598482974568515 #percentage of samples used per tree 
gamma = 3.3681575402083705
rate = 0.19662648468513327
pct_features = 0.9236631906984126 #percentage of features used per tree
alpha_val = 128.2243404673071 #L1 regularization on leaf weights

### GB hyperparams
gb_depth = 8
gb_estimators = 50
gb_rate = .4


### Models
tree_models = [
    
    RandomForestRegressor(n_estimators=100, max_depth=24, max_features=75, random_state=0),
        
    xgb.XGBRegressor(booster = learner, objective ='reg:squarederror', 
                          colsample_bytree = pct_features, 
                          learning_rate = rate,
                          max_depth = depth, 
                          alpha = alpha_val, 
                          min_child_weight = 30.780029105065292,
                          n_estimators = estimators,
                          eval_metric = eval_metric_chosen,
                          subsample = subsamples,
                          random_state=0),
    
    GradientBoostingRegressor(n_estimators=gb_estimators,
                                   learning_rate=gb_rate,
                                   max_depth=gb_depth,
                                   random_state=0, loss='ls'),
    
    lgb.LGBMRegressor(boosting_type = 'gbdt',  num_leaves=31, 
                          max_depth= - 1, learning_rate=0.1, 
                          n_estimators=100, subsample_for_bin=200000, 
                          objective=None, class_weight=None, 
                          min_split_gain=0.0, min_child_weight=0.001, 
                          min_child_samples=20, subsample=1.0, subsample_freq=0, 
                          colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
                          random_state=0, n_jobs=- 1, silent=True, 
                          importance_type='split')
]

models_dummy = [
    Lasso(alpha = 8e-6, normalize = True, max_iter = 1000, tol = 1e-5)
    
        
]


############## Run 1st level Models
# Train Encoded
S_train_encoded, S_test_encoded = stacking(tree_models,                   
                           X_train_encoded, np.ravel(y_train), X_test_encoded,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
            
                           metric = metrics.mean_squared_error, 
    
                           n_folds=4, 

                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=0)



## Train One Hot: Set 1
S_train_onehot_1, S_test_onehot_1 = stacking(models_dummy,                   
                           X_train_onehot_1, np.ravel(y_train), X_test_onehot_1,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
            
                           metric = metrics.mean_squared_error, 
    
                           n_folds=4, 
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=0)


## Train One Hot: Set 2
S_train_onehot_2, S_test_onehot_2 = stacking(models_dummy,                   
                           X_train_onehot_2, np.ravel(y_train), X_test_onehot_2,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
            
                           metric = metrics.mean_squared_error, 
    
                           n_folds=4, 
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=0)

# Stack predictions from 1st level models
S_train = np.hstack((S_train_encoded, S_train_onehot_1, S_train_onehot_2))
S_test = np.hstack((S_test_encoded, S_test_onehot_1, S_test_onehot_2))

Training 1st level models


C:\Users\Lejvi\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5822486847337132, tolerance: 0.23712723665948496
  positive)


## 5.2 Meta-Level layer

The stacked predictions from the first layer were appended to our data again. In this layer, we turned all categorical variables into strings, and tried to spoil CATBOOST algorithm’s ability to handle categorical variables. The dataset we fed into CATBOOST consisted of the original data (with string
categorical variables) and the six columns coming from the 1st level stacked predictions. We used
an off-the-shelf parameterization, and fine tuned the number of iterations, the learning rate, and the
depth of the algorithm. We appended our data to this layer at the hopes that CATBOOST would capture
some features of the categorical nature of our data that label encoding and OneHot encoding may have
missed.

In [23]:
# Meta level
print('Training Meta Model on stacked predictions from 1st level and the data')

# Cat Boost implementation: requires conda install -c conda-forge catboost

#########################################################################
################### Making data categorical first to feed into cat boost
Train_cat = Train
Test_cat  = Test

Train_cat = Train_cat[vars_all_encoded +['y']]   
Test_cat = Test_cat[vars_all_encoded +['y']]  

for col in vars_dummy_encoded:
    Train_cat.loc[:,col] = Train[col].astype(str)
    Test_cat.loc[:,col] = Test[col].astype(str)
    
# Merge in embedded occ and occly
Train_cat = pd.concat([Train_cat, X_embed_train], axis = 1)
Test_cat = pd.concat([Test_cat, X_embed_test], axis = 1)

X_train_cat = Train_cat.drop("y", axis = 1)
y_train_cat = Train_cat["y"]    
    
X_test_cat = Test_cat.drop("y", axis = 1)
y_test_cat = Test_cat["y"]      


############# Append original data to predictions from 1st level
S_train_append = pd.DataFrame(S_train, columns = ['predictor' + str(x) for x in range(1,S_train.shape[1]+1)])
S_test_append = pd.DataFrame(S_test, columns = ['predictor' + str(x) for x in range(1,S_train.shape[1]+1)])

X_train_cat_append = pd.concat((X_train_cat,S_train_append), axis = 1)
X_test_cat_append = pd.concat((X_test_cat,S_test_append), axis = 1)

#####################################################################
############## Meta Algorithm: CatBoost
cb = CatBoostRegressor(loss_function = 'RMSE', random_seed= 0, iterations=3000, learning_rate= .01, depth= 10, verbose = 0)

cb.fit(X_train_cat_append,y_train_cat)

y_pred_app = cb.predict(X_test_cat_append)

Training Meta Model on stacked predictions from 1st level and the data


C:\Users\Lejvi\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# 6. Model Results

The algorithm was trained
using only the training sample with log wages as the target variable. The results show the error on
the test sample using the three metrics required for the competition:

**RMS-log:** 0.4774074678408843 

**RMS-level:** 22488.752597078223

**AbsDev-log:** 3.165844131187269

The RMS-log for the Mincer Regression is 0.74 for our data. Our algorithm thus improves RMS-log by at least $35\%$ relative to the Mincer Regression model of earnings.

In [24]:
print('RMSE on test data is:', np.sqrt(metrics.mean_squared_error(np.ravel(y_test), y_pred_app)))

print('RMSE on test data using levels is:', np.sqrt(metrics.mean_squared_error(np.ravel(np.exp(y_test)), np.exp(y_pred_app))))

print('Absolute Deviation on test data is:', np.max(np.abs(np.ravel(y_test) - y_pred_app)))


RMSE on test data is: 0.4774074678408843
RMSE on test data using levels is: 22488.752597078223
Absolute Deviation on test data is: 3.165844131187269
